In [35]:
#
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
# Load Major Salivary Tumor Listing data
salivary_df = pd.read_csv("MajorSalivaryListing.csv",low_memory=False)

In [33]:
# Look at data header
display(salivary_df.head())
display(salivary_df.info())



,Survival months,RX Summ--Surg Prim Site (1998+),Age recode with <1 year olds,"Race recode (W, B, AI, API)",Reason no cancer-directed surgery,RX Summ--Scope Reg LN Sur (2003+),First malignant primary indicator,Histologic Type ICD-O-3,Site recode ICD-O-3/WHO 2008,Histology recode - broad groupings,...,LN Head and Neck Levels VI-VII Recode (2010+),LN Head and Neck Other Recode (2010+),Lymph Node Size Recode (2010+),LN Positive Axillary Level I-II Recode (2010+),Major Vein Involvement Recode (2010+),Measured Basal Diameter Recode (2010+),Measured Thickness Recode (2010+),Mitotic Rate Melanoma Recode (2010+),Primary by international rules,SEER registry (with CA and GA as whole states)
0,14,38,75-79 years,White,Surgery performed,4 or more regional lymph nodes removed,No,8041,Salivary Gland,"8010-8049: epithelial neoplasms, NOS",...,Blank(s),Blank(s),No involved regional nodes,Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Yes,California
1,86,30,85+ years,White,Surgery performed,Blank(s),No,8071,Salivary Gland,8050-8089: squamous cell neoplasms,...,Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Yes,California
2,188,27,70-74 years,Asian or Pacific Islander,Surgery performed,NaN,No,9699,NHL - Extranodal,9670-9699: nhl - mature b-cell lymphomas,...,Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Yes,California
3,25,0,65-69 years,White,Not recommended,NaN,No,8200,Salivary Gland,8140-8389: adenomas and adenocarcinomas,...,Blank(s),Blank(s),No involved regional nodes,Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Yes,California
4,30,42,55-59 years,Asian or Pacific Islander,Surgery performed,4 or more regional lymph nodes removed,Yes,8502,Salivary Gland,8500-8549: ductal and lobular neoplasms,...,Blank(s),Blank(s),No involved regional nodes,Blank(s),Blank(s),Blank(s),Blank(s),Blank(s),Yes,California


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28030 entries, 0 to 28029
Data columns (total 85 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   Survival months                                          28030 non-null  object
 1   RX Summ--Surg Prim Site (1998+)                          28030 non-null  int64 
 2   Age recode with <1 year olds                             28030 non-null  object
 3   Race recode (W, B, AI, API)                              28030 non-null  object
 4   Reason no cancer-directed surgery                        28030 non-null  object
 5   RX Summ--Scope Reg LN Sur (2003+)                        17943 non-null  object
 6   First malignant primary indicator                        28030 non-null  object
 7   Histologic Type ICD-O-3                                  28030 non-null  int64 
 8   Site recode ICD-O-3/WHO 2008        

None